<div id="singlestore-header" style="display: flex; background-color: rgba(209, 153, 255, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/notes.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Setup CDC Replication from MongoDB® to SingleStore using SQL commands</h1>
    </div>
</div>

<table style="border: 0; border-spacing: 0; width: 100%; background-color: #03010D"><tr>
    <td style="padding: 0; margin: 0; background-color: #03010D; width: 33%; text-align: center"><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-vertical.png" style="height: 200px;"/></td>
    <td style="padding: 0; margin: 0; width: 66%; background-color: #03010D; text-align: right"><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/notebooks/cdc-mongodb-sql-commands/images/cdc-in-header.png" style="height: 250px"/></td>
</tr></table>

## When do you use SingleStore's native replication capability from MongoDB ? 

SingleStore's native data replication gives you the ability to do one-time snapshot, or continuous change data capture CDC from MongoDB® to SingleStoreDB. This provides a quick and easy way to replicate data and power up analytics on MongoDB® data.

## What you will learn in this notebook:

Replicate MongoDB® collections to SingleStore
1. Directly without transformations
2. Flattening required fields into columns of a table
3. Normalizing collection into multiple tables 


## 1. Replicate directly without transformations

To replicate the required collections, provide the list of collections using `"collection.include.list": "<Collection list>"` at the time of link creation, the parameter takes a comma-separated list of regular expressions that match collection names (in databaseName.collectionName format) 

In [50]:
%%sql
DROP DATABASE IF EXISTS sample_analytics;
CREATE DATABASE sample_analytics;

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> Make sure to select a database from the drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt>  to connect to that database.</p>
    </div>
</div>

In [54]:
%%sql
CREATE LINK cdclink AS MONGODB
CONFIG '{"mongodb.hosts":"ac-t7n47to-shard-00-00.tfutgo0.mongodb.net:27017,ac-t7n47to-shard-00-01.tfutgo0.mongodb.net:27017,ac-t7n47to-shard-00-02.tfutgo0.mongodb.net:27017",
"collection.include.list": "sample_analytics.customers",
"mongodb.ssl.enabled":"true",
"mongodb.authsource":"admin",
"mongodb.members.auto.discover": "true"    
    }'
CREDENTIALS '{
    "mongodb.user":"mongo_sample_reader",
    "mongodb.password":"SingleStoreRocks27017"
    }'

++
||
++
++

Check if the link got created

In [55]:
%%sql
SHOW LINKS;

Link,Type,Description
cdclink,MONGODB,


The following step automatically creates the required tables and pipelines on SingleStoreDB for every collection configured for replication

In [56]:
%%sql
CREATE TABLES AS INFER PIPELINE AS LOAD DATA LINK cdclink '*' FORMAT AVRO;

++
||
++
++

Start pipelines to begin replicating the data 

In [57]:
%%sql
START ALL PIPELINES;

++
||
++
++

In [58]:
%%sql
SHOW TABLES;

Tables_in_sample_analytics
customers


The customer collection from MongoDB are replicated into SingleStore in the default format of _id and _more BSON columns that are compatible with Kai API

In [64]:
%%sql
SELECT (_id :> JSON),(_more :> JSON) FROM customers LIMIT 2;

(_id :> JSON),(_more :> JSON)
{'$oid': '5ca4bbcea2dd94ee58162af3'},"{'accounts': [929644, 937618, 881424], 'address': '8077 Mendoza Pines\nWalterfurt, KS 81392', 'birthdate': {'$date': {'$numberLong': '-37139819000'}}, 'email': 'christopher91@hotmail.com', 'name': 'John Guzman', 'tier_and_details': {'400aa61d323a492f9688b99a8ada975c': {'active': True, 'benefits': ['financial planning assistance', 'travel insurance'], 'id': '400aa61d323a492f9688b99a8ada975c', 'tier': 'Silver'}}, 'username': 'edwardszachary'}"
{'$oid': '5ca4bbcea2dd94ee58162b40'},"{'accounts': [937558, 141237], 'address': '2381 Brooks Union\nWest Margaretburgh, NJ 12668', 'birthdate': {'$date': '1970-09-26T09:30:35.000Z'}, 'email': 'anna26@gmail.com', 'name': 'Bryan Huff', 'tier_and_details': {}, 'username': 'fletchernicole'}"


<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>

## 2. Flattening required fields from document into columns
CDC replication also gives additional flexibility to define your own table structure at SingleStore as you bring in data from MongoDB collections. In the following examples data from MongoDB collections are transformed when brought to SingleStoreDB

Fields like `username`, `name`, `email` are flattened into columns of the table and rest of the document is stored in _more column.
The following commands create a table, a stored procedure and a pipeline required for the data replication 

In [66]:
%%sql
CREATE TABLE `customers_flattened` (
  `_id` bson NOT NULL,
  `username` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `name` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `email` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `_more` bson NOT NULL COMMENT 'KAI_MORE' ,
  `$_id` as BSON_NORMALIZE_NO_ARRAY(`_id`) PERSISTED longblob COMMENT 'KAI_AUTO' ,
  SHARD KEY `__SHARDKEY` (`$_id`),
  UNIQUE KEY `__PRIMARY` (`$_id`) USING HASH,
  SORT KEY `__UNORDERED` ()
) 

++
||
++
++

In [67]:
%%sql
CREATE OR REPLACE PROCEDURE `customers_apply_changes`(changes query(`__operation` int(11) NOT NULL, `_id` longblob NOT NULL, `_more` longblob NOT NULL))
RETURNS void AS 
DECLARE rowsDeleted INT;
BEGIN REPLACE INTO  `sample_analytics`.`customers_flattened` SELECT `_id`:>BSON AS `_id`, BSON_EXTRACT_STRING(`_more`,'username') AS `username`, BSON_EXTRACT_STRING(`_more`,'name') AS `name`, BSON_EXTRACT_STRING(`_more`,'email') AS `email`,
BSON_EXCLUDE_MASK(`_more`,'{"_id": 1,"username": 1,"name": 1,"email": 1}') AS `_more`FROM changes WHERE __operation != 1;
SELECT count(*) INTO rowsDeleted FROM changes WHERE changes.__operation = 1;
IF rowsDeleted > 0 THEN
DELETE dest FROM `sample_analytics`.`customers_flattened` AS dest INNER JOIN changes ON dest.`$_id` = BSON_NORMALIZE_NO_ARRAY(changes.`_id`) WHERE changes.__operation = 1; END IF;
END;

++
||
++
++

In [68]:
%%sql
CREATE AGGREGATOR PIPELINE `customers_apply_changes`
AS LOAD DATA LINK cdclink 'customers'
BATCH_INTERVAL 2500
MAX_PARTITIONS_PER_BATCH 1
DISABLE OFFSETS METADATA GC
REPLACE
KEY(`_id`)
INTO PROCEDURE `customers_apply_changes`
FORMAT AVRO
(
    __operation <- `__operation`,
    _id <- `payload`::`_id`,
    _more <- `payload`::`_more`
)

++
||
++
++

In [69]:
%%sql
START ALL PIPELINES;

++
||
++
++

In [70]:
%%sql
SHOW TABLES;

Tables_in_sample_analytics
customers
customers_flattened


In [72]:
%%sql
SELECT _id :> JSON,username, name, email, _more :> JSON FROM customers_flattened LIMIT 10;

_id :> JSON,username,name,email,_more :> JSON
{'$oid': '5ca4bbcea2dd94ee58162b99'},christophercooper,Denise Curtis,fmoore@hotmail.com,"{'accounts': [385025, 412109, 201161], 'address': '942 Jennifer Forest\nWatkinsburgh, MI 96943', 'birthdate': {'$date': '1975-03-06T20:46:53.000Z'}, 'tier_and_details': {'28cf2d66a5944fa8a4c0ac9725ad1b62': {'active': True, 'benefits': ['shopping discounts', 'concert tickets'], 'id': '28cf2d66a5944fa8a4c0ac9725ad1b62', 'tier': 'Silver'}, '921b3281d79d4f818be9906c5fa8e43f': {'active': True, 'benefits': ['travel insurance', 'financial planning assistance'], 'id': '921b3281d79d4f818be9906c5fa8e43f', 'tier': 'Gold'}}}"
{'$oid': '5ca4bbcea2dd94ee58162af5'},xrodriguez,Johnathan Reyes,wnguyen@gmail.com,"{'accounts': [354107], 'address': '8294 Hudson Stream Suite 944\nSamanthafurt, OK 50810', 'birthdate': {'$date': '1972-06-21T09:28:56.000Z'}, 'tier_and_details': {}}"
{'$oid': '5ca4bbcea2dd94ee58162af3'},edwardszachary,John Guzman,christopher91@hotmail.com,"{'accounts': [929644, 937618, 881424], 'address': '8077 Mendoza Pines\nWalterfurt, KS 81392', 'birthdate': {'$date': {'$numberLong': '-37139819000'}}, 'tier_and_details': {'400aa61d323a492f9688b99a8ada975c': {'active': True, 'benefits': ['financial planning assistance', 'travel insurance'], 'id': '400aa61d323a492f9688b99a8ada975c', 'tier': 'Silver'}}}"
{'$oid': '5ca4bbcea2dd94ee58162b40'},fletchernicole,Bryan Huff,anna26@gmail.com,"{'accounts': [937558, 141237], 'address': '2381 Brooks Union\nWest Margaretburgh, NJ 12668', 'birthdate': {'$date': '1970-09-26T09:30:35.000Z'}, 'tier_and_details': {}}"
{'$oid': '5ca4bbcea2dd94ee58162b50'},charlotte30,Emily Davidson,wintersmorgan@hotmail.com,"{'accounts': [770262, 759997], 'address': '238 Romero Hollow Apt. 868\nPort Joshuaville, TX 07164', 'birthdate': {'$date': '1973-01-26T01:24:35.000Z'}, 'tier_and_details': {}}"
{'$oid': '5ca4bbcea2dd94ee58162c2f'},linda08,Christopher Smith,benjaminmoore@yahoo.com,"{'accounts': [190959, 520970, 461793, 141474, 805151], 'address': '900 Brett Terrace\nWest Deborah, WV 45028', 'birthdate': {'$date': '1990-01-25T17:10:29.000Z'}, 'tier_and_details': {'6bf9633c0fa34014b85383ae07619dce': {'active': True, 'benefits': ['24 hour dedicated line', 'shopping discounts'], 'id': '6bf9633c0fa34014b85383ae07619dce', 'tier': 'Gold'}}}"
{'$oid': '5ca4bbcea2dd94ee58162a86'},ethanarias,Christopher Watson,lscott@gmail.com,"{'accounts': [884849], 'address': '196 Baker Key\nNorth Billystad, AL 12800', 'birthdate': {'$date': '1984-07-24T10:14:22.000Z'}, 'tier_and_details': {}}"
{'$oid': '5ca4bbcea2dd94ee58162bb4'},rodney14,Jaime Downs,stevenlucero@yahoo.com,"{'accounts': [164260, 266920], 'address': '613 Jamie Canyon\nCristinashire, AK 36686', 'birthdate': {'$date': '1985-03-11T07:11:53.000Z'}, 'tier_and_details': {'6b99c1ac3c70417daad76d6ea6b72c71': {'active': True, 'benefits': ['car rental insurance'], 'id': '6b99c1ac3c70417daad76d6ea6b72c71', 'tier': 'Silver'}, '874b68cf6712432fb8cdec8b005e0a57': {'active': True, 'benefits': ['financial planning assistance', 'sports tickets'], 'id': '874b68cf6712432fb8cdec8b005e0a57', 'tier': 'Silver'}}}"
{'$oid': '5ca4bbcea2dd94ee58162c2c'},christopherallen,Ronald Curry,millerbrandy@yahoo.com,"{'accounts': [448816, 638191], 'address': '59101 Saunders Mountain\nTravismouth, ID 50827', 'birthdate': {'$date': '1988-03-22T20:34:28.000Z'}, 'tier_and_details': {}}"
{'$oid': '5ca4bbcea2dd94ee58162b31'},wwilson,Jonathon Small,michael32@hotmail.com,"{'accounts': [296866, 316726], 'address': '891 Golden Green\nStephenshire, ND 88063', 'birthdate': {'$date': '1981-08-10T18:21:48.000Z'}, 'tier_and_details': {}}"


## 3. Normalize a collection into multiple tables 
In the following example a collection of MongoDB is normalized into two different tables on SingleStore.

In [80]:
%%sql
DROP DATABASE IF EXISTS sample_airbnb;
CREATE DATABASE sample_airbnb;

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> Make sure to select a database from the drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt>  to connect to that database.</p>
    </div>
</div>

In [84]:
%%sql
CREATE LINK source_listingsAndReviews AS MONGODB
CONFIG '{"mongodb.hosts":"ac-t7n47to-shard-00-00.tfutgo0.mongodb.net:27017,ac-t7n47to-shard-00-01.tfutgo0.mongodb.net:27017,ac-t7n47to-shard-00-02.tfutgo0.mongodb.net:27017",
"collection.include.list": "sample_airbnb.*",
"mongodb.ssl.enabled":"true",
"mongodb.authsource":"admin",
"mongodb.members.auto.discover": "true"    
    }'
CREDENTIALS '{
    "mongodb.user":"mongo_sample_reader",
    "mongodb.password":"SingleStoreRocks27017"
    }'

++
||
++
++

In [85]:
%%sql
SHOW LINKS;

Link,Type,Description
source_listingsAndReviews,MONGODB,


In [86]:
%%sql
CREATE TABLE `listings` (
  `_id` BSON NOT NULL,
  `name` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `access` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `accommodates` int(11) DEFAULT NULL,
  `_more` BSON NOT NULL,
  `$_id` as BSON_NORMALIZE_NO_ARRAY(`_id`) PERSISTED longblob,
  SHARD KEY `__SHARDKEY` (`$_id`),
  UNIQUE KEY `__PRIMARY` (`$_id`) USING HASH,
  SORT KEY `__UNORDERED` ()
) 

++
||
++
++

In [87]:
%%sql
CREATE TABLE `reviews` (
  `listingid` BSON NOT NULL,
  `review_scores_accuracy` int(11) DEFAULT NULL,
  `review_scores_cleanliness` int(11) DEFAULT NULL,
  `review_scores_rating` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  `$listingid` as BSON_NORMALIZE_NO_ARRAY(`listingid`) PERSISTED longblob,
  SHARD KEY `__SHARDKEY` (`$listingid`),
  UNIQUE KEY `__PRIMARY` (`$listingid`) USING HASH,
  SORT KEY `__UNORDERED` ()
) 

++
||
++
++

In [88]:
%%sql
CREATE OR REPLACE PROCEDURE `listingsAndReviews_apply_changes`(changes query(`__operation` int(11) NOT NULL, `_id` longblob NOT NULL, `_more` longblob NOT NULL))
RETURNS void AS 
DECLARE rowsDeleted INT;
BEGIN 
    
REPLACE INTO `listings` SELECT `_id`:>BSON AS `_id`, BSON_EXTRACT_STRING(`_more`,'name') AS `name`, BSON_EXTRACT_STRING(`_more`,'access') AS `access`,
BSON_EXTRACT_BIGINT(`_more`,'accommodates') AS `accommodates`, BSON_EXCLUDE_MASK(`_more`,'{"_id": 1,"name": 1,"review_scores": 1,"access" : 1, "accommodates" : 1}') AS `_more`
FROM changes WHERE __operation != 1;

REPLACE INTO `reviews` SELECT `_id`:>BSON AS `listingid`, BSON_EXTRACT_BIGINT(`_more`,'review_scores','review_scores_accuracy') AS `review_scores_accuracy`,
BSON_EXTRACT_BIGINT(`_more`,'review_scores','review_scores_cleanliness') AS `review_scores_cleanliness`, BSON_EXTRACT_BIGINT(`_more`,'review_scores','review_scores_rating') AS `review_scores_rating`
FROM changes WHERE __operation != 1;

SELECT count(*) INTO rowsDeleted FROM changes WHERE changes.__operation = 1;
IF rowsDeleted > 0 THEN
DELETE dest FROM `listings` AS dest INNER JOIN changes ON dest.`$_id` = BSON_NORMALIZE_NO_ARRAY(changes.`_id`) WHERE changes.__operation = 1; 
DELETE dest FROM `reviews` AS dest INNER JOIN changes ON dest.`$listingid` = BSON_NORMALIZE_NO_ARRAY(changes.`_id`) WHERE changes.__operation = 1; 
END IF;

END;

++
||
++
++

In [89]:
%%sql
CREATE AGGREGATOR PIPELINE `listingsAndReviews`
AS LOAD DATA LINK source_listingsAndReviews 'sample_airbnb.listingsAndReviews'
BATCH_INTERVAL 2500
MAX_PARTITIONS_PER_BATCH 1
DISABLE OFFSETS METADATA GC
REPLACE
KEY(`_id`)
INTO PROCEDURE `listingsAndReviews_apply_changes`
FORMAT AVRO
(
    __operation <- `__operation`,
    _id <- `payload`::`_id`,
    _more <- `payload`::`_more`
)

++
||
++
++

In [91]:
%%sql
START ALL PIPELINES;

++
||
++
++

In [93]:
%%sql
SHOW TABLES;

Tables_in_sample_airbnb
listings
reviews


In [94]:
%%sql
SELECT _id:>JSON ,name, access, accommodates FROM listings LIMIT 10;

_id:>JSON,name,access,accommodates
10084023,City center private room with bed,"Living Room , Kitchen and Toilet, All cooking equipment can be used too",1
10527243,Tropical Jungle Oasis,,4
10992286,Holoholo Inn: Rain Forest (Priv-2),Guest access to: Common Room with TV; Full Kitchen; Free WiFi.,2
11567997,The Local Nook // Cozy Kona Hale,"The main entrance is to the right through the white gate in front of your parking stall. We ask that guest please enter this way to give our parents and yourselves some extra privacy, though there is another entry door to the left in front of the main house entrance.",2
12552675,The Porto Concierge - White Martin,Guests have access to all the amenities of the apartment.,3
1176693,BEST REVIEWS*BEST MALLS*SAFE STAY*DIMSUM*CWB*MTR,"The entire apartment with all amenities is accessible for my guests. All closets are empty but full of hangers, you will find toiletries and hair dryers in the bathrooms. the kitchen is fully equipped with pots, pans, china, glasses and cutlery. Basics like tea, coffee, water, milk, and capsules for the Nespresso Machine are provided to make sure your arrival will be relaxed. Detergents for washing machine and dish washer are provided for a stay of 3 nights. My stand-by guest manager will happily provide more supplies on request for small additional costs.",8
11007058,Alcam Colón 42 Apartment,,4
12183522,so homey and spacious and comfy room!,"Easy access to public transport, main road is just across to where building is located. Car park is also available, on hourly to day rate.",3
13488308,Spacious & Cosy 1BR apt in Gramercy with balcony!,"Guests are provided complete privacy for the duration of their stay, however, in order to ensure a superior guest experience, I am available to address your needs 24 hours a day 7 days a week. If you have a question or concern, I encourage you to contact me by message. I am dedicated to making your stay the best possible, and I am happy to address your needs, whatever they may be.",2
1719942,Studio WBwGG,"Our guests will have, for their own, a completely independent basement, sharing with us, just the access from the front door. Remark: The ceiling has wood bars on each 1 meter and it's not vey tall; it will not be very comfortable to walk for persons taller than 1.80 mts. The wide and beautiful garden is reached by the basement and, besides we also have acces to it, during guest stays, we will do our best to become invisible. The bathroom is exclusively for the guests. The sofa-bed is huge and very comfortable. The bar is equipped with fridge, microwave, espresso&capppuccino machine, toaster and sink. On tv, there dozens of cable channels, pay-per-view access and game console. Wi-fi with internet access is coveres in the full area, including the garden. Also available for use; foosball, drum, guitars, computer, books and magazines.",2


In [96]:
%%sql
SELECT listingid:>JSON, review_scores_accuracy,review_scores_cleanliness, review_scores_rating FROM reviews LIMIT 10;

listingid:>JSON,review_scores_accuracy,review_scores_cleanliness,review_scores_rating
32528492,None,None,None
31913927,None,None,None
32338886,None,None,None
5814254,10,10,98
5923031,10,10,98
32481559,10,10,100
4373803,None,None,None
31302323,None,None,None
961596,9,9,92
8358372,10,10,90
